# EV Charger Finder


## Installs

In [ ]:
!pip install haversine
!pip install pandas

## Imports

In [ ]:
import pandas as pd
from haversine import haversine, Unit
import requests

## Data Set

In [ ]:
file_path = ''
df = pd.read_csv(file_path)

In [ ]:
file_path = ''
df = pd.read_excel(file_path)

In [ ]:
df['Address'] = df.apply(lambda row: f"{row['Street Address']}, {row['City']}, {row['State']} {row['ZIP']}", axis=1)
df = df.drop(columns=['Street Address', 'City', 'State', 'ZIP'])

In [ ]:
df["EV Connector Types"].unique()

In [ ]:
df.to_csv("",index=False)

In [ ]:
df.columns.tolist()

In [ ]:
# Let's get all the column names to identify the ones for latitude, longitude, and EV connector types
column_names = df.columns.tolist()
column_names

## Haversine Distance Calculation

In [ ]:
# Assuming 'Groups With Access Code' might contain connector type information
# For the purpose of this example, let's filter stations that are accessible to the public, as a placeholder
# for actual connector type matching (which would require the appropriate column)

s_lat = 42.3528049
s_lng = -83.0642518


# Filter for public stations as a placeholder for connector type filtering
public_stations = df[df['Groups With Access Code'].str.contains('Public', na=False)]

# Example user input for their current location (s_lat, s_lng)
user_location = (s_lat, s_lng)  # Example: Downtown Los Angeles, CA

# Function to calculate distance between two points
def calculate_distance(user_loc, station_loc):
    return haversine(user_loc, station_loc, unit=Unit.MILES)

# Add a new column 'Distance' to the dataframe by applying the calculate_distance function
public_stations['Distance'] = public_stations.apply(
    lambda row: calculate_distance(user_location, (row['Latitude'], row['Longitude'])), axis=1
)

# Sort the dataframe by the new 'Distance' column to find the nearest station
nearest_stations = public_stations.sort_values(by='Distance')

# Display the top 5 nearest stations
nearest_stations[['Station Name', 'Street Address', 'City', 'State', 'ZIP', 'Latitude','Longitude','Distance','EV Connector Types']].head(5)

In [ ]:
res = nearest_stations[['Station Name', 'Street Address', 'City', 'State', 'ZIP', 'Latitude','Longitude','Distance','EV Connector Types']].head(5)
res['Address'] = nearest_stations.apply(lambda row: f"{row['Street Address']}, {row['City']}, {row['State']} {row['ZIP']}", axis=1)
res = res.drop(columns=['Street Address', 'City', 'State', 'ZIP'])
res = res.reset_index(drop=True)

In [ ]:
res

## OSRM API Testing

In [ ]:
lat_lng_list = res[['Latitude', 'Longitude']].itertuples(index=False, name=None)

q_string = ';'.join([f"{lng},{lat}" for lat, lng in lat_lng_list])
url = f"http://router.project-osrm.org/table/v1/driving/{float(s_lng)},{float(s_lat)};{q_string}?sources=0&annotations=duration,distance"

In [ ]:
url

In [ ]:
r = requests.get(url)
response = r.json()
res['Duration'] = response['durations'][0][1:]
res['Distance'] = response['distances'][0][1:]

In [ ]:
res

In [ ]:
res = res.sort_values(by="Duration").reset_index(drop=True)

In [ ]:
def convert_seconds_to_hms(seconds):
  hours = seconds // 3600
  seconds %= 3600
  minutes = seconds // 60
  seconds %= 60

  if hours > 0:
    return f"{hours:.0f} hr {minutes:.0f} min"
  elif minutes > 0:
    return f"{minutes:.0f} min"
  else:
    return f"1 min"

# Example usage
duration_in_seconds = 12345
duration_string = convert_seconds_to_hms(duration_in_seconds)
print(f"{duration_string}")

In [ ]:
res["Duration"] = res["Duration"].apply(lambda dur:convert_seconds_to_hms(dur))

In [ ]:
def meters_to_miles(meters):
  miles = meters / 1609.34
  return miles

# Example usage
distance_in_meters = 1000
distance_in_miles = meters_to_miles(distance_in_meters)
print(f"{distance_in_meters} meters is equal to {distance_in_miles:.2f} miles.")

In [ ]:
res["Distance"] = res["Distance"].apply(lambda dis:f"{round(meters_to_miles(dis),2)} miles")

In [ ]:
res

In [ ]:
res["URL"] = res.apply(lambda row: f"https://www.google.com/maps/dir/?api=1&origin={float(s_lat)},{float(s_lng)}&destination={row["Latitude"]},{row["Longitude"]}&travelmode=driving&dir_action=navigate",axis=1)

In [ ]:
res = res.drop(columns=['Latitude', 'Longitude'])

In [ ]:
res

In [ ]:
stations = res.to_dict(orient='records')

In [ ]:
stations